In [ ]:
import jax
import jax.numpy as jnp
from   jax import vmap
from   jax import jit
from   jax import jacfwd, jacrev
from   jax import grad, jvp
from   jax import random as jrd

import numpy as np
from numpy.linalg import norm

import matplotlib.pyplot as plt
from scipy.optimize import minimize
from scipy.optimize import least_squares
from matplotlib import cm
from matplotlib.ticker import LinearLocator
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from random import sample
from time import time
import math
import random

import os
import scipy.io
import sympy as sym
#For the animation
from matplotlib.animation import FuncAnimation
from matplotlib import rc
from IPython.display import HTML

from numpy.polynomial import Polynomial
from scipy.sparse import csr_matrix
from scipy.integrate import dblquad, quad
from scipy import integrate

key = jrd.PRNGKey(42)
key1, key2, key3, key4 = jrd.split(key, num = 4)
dim = 2
'Data'
rho = 1.043
mu = 0.0035
#Re = 100. #rho/mu
n_pts = 81 # 71 41, 51,
n_pts_hat = 81

n_x = n_pts   #number of points in the x direction
n_y = n_pts   #number of points in the y direction

n_x_hat = n_pts_hat
n_y_hat = n_pts_hat

deltax, deltay = 1./n_x, 1./n_y
R = 1.
x_lb, y_lb  = 0, 0
x_ub, y_ub  = R, R
T0, T1 = 0., 1.
#N = 100
#dt = (T1 - T0)/n_pts
minval = - np.sqrt(6)/np.sqrt(dim + 1)
maxval =   np.sqrt(6)/np.sqrt(dim + 1)

In [ ]:
#n_xy = n_x*n_y
x_gauss, wx_gauss = np.polynomial.legendre.leggauss(n_x)
y_gauss, wy_gauss = np.polynomial.legendre.leggauss(n_y)
xin_mesh, yin_mesh = np.meshgrid(x_gauss, y_gauss)
xin = 0.5*(xin_mesh.T.flatten().T + 1.)
yin = 0.5*(yin_mesh.T.flatten().T + 1.)
poids = 0.25*(wx_gauss*wy_gauss[:,None]).ravel()

xin_hat, yin_hat = xin, yin

In [ ]:

"Boundary points"
xbdry_up = np.linspace(x_lb, x_ub, 4*n_x)
ybdry_up = np.linspace(y_ub, y_ub, 4*n_x)

xbdry_down = xbdry_up
ybdry_down = np.linspace(y_lb, y_lb, 4*n_x)

xbdry_left = np.linspace(x_lb, x_lb, 4*n_y)
ybdry_left = np.linspace(y_lb, y_ub, 4*n_y)

xbdry_right = np.linspace(x_ub, x_ub, 4*n_y)
ybdry_right = ybdry_left

xbdry = np.concatenate([xbdry_up, xbdry_down, xbdry_left, xbdry_right], 0)
ybdry = np.concatenate([ybdry_up, ybdry_down, ybdry_left, ybdry_right], 0)

fig, ax = plt.subplots(1, 2, figsize=(12, 5))
ax[0].scatter(xin, yin, s = 1., marker = 'o', color = 'black', label = "Training points")
#ax[0].scatter(xbdry, ybdry, s = 1., marker = 'o', color = 'r', label = "Boundary points")
ax[1].scatter(xin_hat, yin_hat, s = 1., marker = 'o', color = 'b', label = "Centers of RBFs")
#plt.legend(['Traning points', "Centers of RBF"])
ax[0].legend()
ax[1].legend()
plt.show()


In [ ]:
def test(x, y):
  m, n = 20, 20
  return  jnp.cos(m*x)*jnp.sin(n*y)  #(1. + x + x**3 + x*y**2)*(np.exp(x**2 + y**2)**2 + y*np.sin(5*x))

print("True integral: %.12f" %integrate.nquad(test, [[0., 1.], [0., 1.]])[0])

print("Approximated integral: %.12f" %jnp.dot(poids, test(xin, yin)))

In [ ]:
kx, ky = 4, 4  # ie  x = kx*deltax, y = ky*deltay
h =   10**(-5)
eps = - jnp.log(h)/(kx**2*deltax**2 + ky**2*deltay**2)
print("eps = %f"  %eps)

def RBF(x, y):
  r_square = x**2 + y**2
  return  jnp.exp(- eps*r_square)
  #return r_square**4

In [ ]:

@jit
def u1b(x, y):
    h = 0.0
    t0 = x**2   + (1+h-y)**2
    t1 = (1-x)**2 + (1+h-y)**2
    return y*x*(1-x)*(1./jnp.sqrt(t0) + 1./jnp.sqrt(t1))

n_hat = len(xin_hat)

params     =    np.random.normal(0., 1., size=(2*n_hat,))

def u1(params_u, x, y):

  PHI = RBF(x - xin_hat, y - yin_hat)

  index1 = 0
  index2 = index1 + n_hat

  W = params_u[index1:index2]

  z = jnp.dot(W, PHI)
  return z*x*y*(R - x)*(R - y) + u1b(x, y)

def u2(params, x, y):

  PHI = RBF(x - xin_hat, y - yin_hat)

  index1 = n_hat
  index2 = index1 + n_hat

  W = params[index1:index2]

  z =  jnp.dot(W, PHI)

  return z*x*y*(R - x)*(R - y)


In [ ]:
#u1
u1_x1   = jit(grad(u1,    1))
u1_x1x1 = jit(grad(u1_x1, 1))
u1_x2   = jit(grad(u1,    2))
u1_x2x2 = jit(grad(u1_x2, 2))
u1_x1x2 = jit(grad(u1_x1, 2))

#u2
u2_x1   = jit(grad(u2,    1))
u2_x1x1 = jit(grad(u2_x1, 1))
u2_x2   = jit(grad(u2,    2))
u2_x2x2 = jit(grad(u2_x2, 2))
u2_x1x2 = jit(grad(u2_x1, 2))

#Vectoriser
#u1
u1_vect      = vmap(u1,          (None, 0, 0) )
u1_x1_vect   = vmap(u1_x1,       (None, 0, 0) )
u1_x2_vect   = vmap(u1_x2,       (None, 0, 0) )
u1_x1x1_vect = vmap(u1_x1x1,     (None, 0, 0) )
u1_x1x2_vect = vmap(u1_x1x2,     (None, 0, 0) )
u1_x2x2_vect = vmap(u1_x2x2,     (None, 0, 0) )
#u2
u2_vect      = vmap(u2,          (None, 0, 0) )
u2_x1_vect   = vmap(u2_x1,       (None, 0, 0) )
u2_x2_vect   = vmap(u2_x2,       (None, 0, 0) )
u2_x1x1_vect = vmap(u2_x1x1,     (None, 0, 0) )
u2_x1x2_vect = vmap(u2_x1x2,     (None, 0, 0) )
u2_x2x2_vect = vmap(u2_x2x2,     (None, 0, 0) )
#p

In [ ]:

def norm_div_L2(params):
  eq_div  = u1_x1_vect(params, xin, yin) +  u2_x2_vect(params, xin, yin)
  return  jnp.sqrt(jnp.dot(poids, eq_div**2))

#grad_loss_div = jit(grad(loss_div, 0))

norm_div_L2(params)

In [ ]:
Re = 10000
dt = 0.07

test3: BDF2 + Crank Nicolson on both the diffusion term and on the augmenterd term
#+ NOT multiply by (2Re)*(2dt): good result. Same as before.

@jit
def lagrangean_NS(params, params0, params1, X0, Y0, X1, Y1):

  inertia_term   = (0.5*3.*(u1_vect(params, xin, yin)**2 + \
                           u2_vect(params, xin, yin)**2)  \
                 -     4.*(u1_vect(params1, X1,  Y1)*\
                           u1_vect(params, xin, yin) +  \
                           u2_vect(params1, X1,  Y1)*\
                           u2_vect(params, xin, yin))\
                 +       (u1_vect(params0, X0,  Y0)*\
                          u1_vect(params, xin, yin) +\
                          u2_vect(params0, X0,  Y0)*\
                          u2_vect(params, xin, yin)))/(2*dt)

  diffusion_term1 = 0.5*\
                   (u1_x1_vect(params, xin, yin)**2 + \
                    u1_x2_vect(params, xin, yin)**2 + \
                    u2_x1_vect(params, xin, yin)**2 + \
                    u2_x2_vect(params, xin, yin)**2)/(2*Re)

  diffusion_term2 = (u1_x1_vect(params1,  xin, yin)* \
                     u1_x1_vect(params,   xin, yin) +\
                     u1_x2_vect(params1,  xin, yin)*  \
                     u1_x2_vect(params,   xin, yin) +\
                     u2_x1_vect(params1,  xin, yin)*  \
                     u2_x1_vect(params,   xin, yin) +\
                     u2_x2_vect(params1,  xin, yin)* \
                     u2_x2_vect(params,   xin, yin))/(2*Re)

  augmented_term1 = 0.25*penalty*(u1_x1_vect(params, xin, yin) +\
                                  u2_x2_vect(params, xin, yin))**2

  augmented_term2 = 0.5*penalty*(u1_x1_vect(params1, xin, yin) +\
                                  u2_x2_vect(params1, xin, yin))*\
                                 (u1_x1_vect(params,  xin, yin) +\
                                  u2_x2_vect(params,  xin, yin))

  pdiv_term      =   pressure*(u1_x1_vect(params, xin, yin) +\
                               u2_x2_vect(params, xin, yin))

  L =  jnp.dot(poids, inertia_term)    +\
       jnp.dot(poids, diffusion_term1)  +\
       jnp.dot(poids, diffusion_term2)  +\
       jnp.dot(poids, augmented_term1)  +\
       jnp.dot(poids, augmented_term2)  -\
       jnp.dot(poids, pdiv_term)
  return L

In [ ]:

# NS
pressure = jnp.ones((len(xin),))
penalty  = 10.
epoch = 20
params0 = 0.*params
params1 = 0.*params
L     =  lagrangean_NS(params,params0, params1, xin, yin, xin, yin)
div   =  norm_div_L2(params)
print("k : Initial, Lagrangean_Stokes: %.12f,  norm_divu_L2: %.12f"  %(L, div))

N = 400
for n in range(2, N + 1):

  print("n = %d" %n)

  # Runge-Kutta Method of order 2 (l = 1.)
  # Compute X^(n-1),n.

  # we compute K1 in both directions
  #K1x =  2*u1_vect(params1, xin, yin) - u1_vect(params0, xin, xin)    #we approach U^n(., .) by 2U^{n-1}(., .) - U^{n-2}(., .)
  #K1y =  2*u2_vect(params1, xin, yin) - u2_vect(params0, xin, yin)

  K1x = u1_vect(params, xin, yin)
  K1y = u2_vect(params, xin, yin)

  #we compute K2 in both direction using K1
  K2x = u1_vect(params1, xin - dt*K1x, yin - dt*K1y)
  K2y = u2_vect(params1, xin - dt*K1x, yin - dt*K1y)

  # We compute the approximation of the position. At previous time n-1 in both direction
  # This is the position at time n-1
  X1 = xin - 0.5*dt*(K1x + K2x)
  Y1 = yin - 0.5*dt*(K1y + K2y)

  #Compute X^(n-2),n
  # using the approximation at time n-1. We compute the approximation at time n-2
  # we compute K1 in both direction for the approximation of X^(n-2),n

  K1x = u1_vect(params1, X1, Y1)
  K1y = u2_vect(params1, X1, Y1)

  # we compute K2 in both direction, for the approximation of X^(n-2),n
  K2x = u1_vect(params0, X1 - dt*K1x, Y1 - dt*K1y)
  K2y = u2_vect(params0, X1 - dt*K1x, Y1 - dt*K1y)
  # Th is is the approximation of the position at time n-2
  X0 = X1 - 0.5*dt*(K1x + K2x)
  Y0 = Y1 - 0.5*dt*(K1y + K2y)

  @jit
  def lagrang_NS(params):
    return lagrangean_NS(params, params0, params1, X0, Y0, X1, Y1)

  grad_lagrang_NS   = jit(grad(lagrang_NS, 0))

  for k in range(epoch):

    L_BFGS_B_NS = minimize(lagrang_NS, params, args=(), method="L-BFGS-B", jac=grad_lagrang_NS,\
                      hess= None, hessp= None,\
                      bounds=None, constraints=(), tol= 10**(-8), callback=None, \
                      options= {"maxiter" : 10, "gtol" : 10**(-8)})

    params_new  = L_BFGS_B_NS.x

    pressure = pressure - (penalty)*(u1_x1_vect(params_new, xin, yin) +   u2_x2_vect(params_new, xin, yin))

    L     = lagrang_NS(params_new)
    div   = norm_div_L2(params_new)
    print("k : %d, Lagrangean_NS: %.12f,   norm_div_L2: %.12f"  %(k, L, div))

    if (params_new == params).all():
      break
    params  = params_new

  params0 = params1
  params1 = params

  '''
  # Plotting of BDF1 at each step
  params_NS = params
  grid_number = 100
  x, y = np.meshgrid(np.linspace(x_lb, x_ub, grid_number), np.linspace(x_lb, x_ub, grid_number) )
  XY = np.vstack((np.ravel(x), np.ravel(y))).T
  X = XY[:, 0]
  Y = XY[:, 1]
  one = jnp.ones((len(X),))
  fig, ax = plt.subplots(1, 1, figsize=(6, 5))
  u1_pred = u1_vect(params_NS, X, Y)
  u2_pred = u2_vect(params_NS, X, Y)
  #ax = plt.axes(xlim=(0, R), ylim=(0, R))
  ax.set_title("NS: Re = %d" %Re)
  z1 = u1_pred.reshape(grid_number, grid_number)
  z2 = u2_pred.reshape(grid_number, grid_number)
  #plt.clabel(cp, inline=True, fontsize=15)
  plt.streamplot(x, y,  z1, z2, density = 3)
  cp = ax.quiver(XY[:, 0], XY[:, 1], u1_pred,  u2_pred,  \
           jnp.sqrt(u1_pred**2+ u2_pred**2), color='g', \
           label = '(u1_pred, u2_pred)')
  xaxis = jnp.linspace(x_lb,  x_ub, int((x_ub - x_lb)/0.1 + 1))
  yaxis =  jnp.linspace(y_lb, y_ub, int((y_ub - y_lb)/0.1 + 1))
  #plt.xticks(xaxis)   # ,[str(i) for i in ticks]
  #plt.yticks(yaxis)
  ax.scatter(xbdry, ybdry, s = 2., marker = 'o', color = 'r')
  plt.colorbar(cp)
  '''


In [ ]:
params_NS = params
grid_number = 100
x, y = np.meshgrid(np.linspace(x_lb, x_ub, grid_number), np.linspace(x_lb, x_ub, grid_number))
XY = np.vstack((np.ravel(x), np.ravel(y))).T
X = XY[:, 0]
Y = XY[:, 1]
one = jnp.ones((len(X),))
fig, ax = plt.subplots(1, 1, figsize=(6, 5))
u1_pred = u1_vect(params_NS, X, Y)
u2_pred = u2_vect(params_NS, X, Y)
#ax = plt.axes(xlim=(0, R), ylim=(0, R))
ax.set_title("NS: Re = %d" %Re)
z1 = u1_pred.reshape(grid_number, grid_number)
s1 = np.array(z1)
z2 = u2_pred.reshape(grid_number, grid_number)
s2 = np.array(z2)
#plt.clabel(cp, inline=True, fontsize=15)
strm = plt.streamplot(x, y,  z1, z2, density = 5, color= np.sqrt(s1**2 + s2**2), cmap = "plasma")
fig.colorbar(strm.lines)
#rainbow, plasma, magma, viridis (normal), inferno
#plt.set_cmap("rainbow")
#cp = ax.quiver(XY[:, 0], XY[:, 1], u1_pred,  u2_pred,  \
 #        jnp.sqrt(u1_pred**2+ u2_pred**2), color='g', \
  #       label = '(u1_pred, u2_pred)')
xaxis = jnp.linspace(x_lb,  x_ub, int((x_ub - x_lb)/0.1 + 1))
yaxis =  jnp.linspace(y_lb, y_ub, int((y_ub - y_lb)/0.1 + 1))
#plt.xticks(xaxis)   # ,[str(i) for i in ticks]
#plt.yticks(yaxis)
ax.scatter(xbdry, ybdry, s = 2., marker = 'o', color = 'r')
#plt.colorbar(cp)